In [1]:
import os
import sys
import numpy as np
import pandas as pd
import json

sys.path.append('../')
sys.path.append('../src/')
sys.path.append('../spell/')

import Reader
import ParamsExtractor3
import DataPreprocessor
import EncodeCommand
import DeepLearningAnomalyDetection2

2024-09-03 13:38:43.749741: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-03 13:38:43.798742: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-03 13:38:44.602863: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# First type of log file:
# laurel_anomalous_new
fp = '../data/linpeas_logs_2_silent_attack/laurel_logs.log'
fps = [fp]
fps += [fp + f'.{str(i)}' for i in range(1,8)]
print(fps)

['../data/linpeas_logs_2_silent_attack/laurel_logs.log', '../data/linpeas_logs_2_silent_attack/laurel_logs.log.1', '../data/linpeas_logs_2_silent_attack/laurel_logs.log.2', '../data/linpeas_logs_2_silent_attack/laurel_logs.log.3', '../data/linpeas_logs_2_silent_attack/laurel_logs.log.4', '../data/linpeas_logs_2_silent_attack/laurel_logs.log.5', '../data/linpeas_logs_2_silent_attack/laurel_logs.log.6', '../data/linpeas_logs_2_silent_attack/laurel_logs.log.7']


In [3]:
fp = fps[0]

data_list = []
lines_limit = 1000 # limit reading lines, done for memory constraints
i = 0

with open(fp, 'r') as file:
    for line in file:
        i+=1
        if i > lines_limit:
            break
            # Find the index of the first '{'
        index = line.find('{')
        if index != -1:
            json_data = line[index:]
            data = json.loads(json_data)
            data_list.append(data)
                
df = pd.DataFrame(data_list)

In [4]:
df.head()

,hostname,ident,msg,severity,facility,timereported
0,localhost,laurel,type=SYSCALL msg=audit(1720451679.133:2225): a...,info,local0,2024-07-09T12:21:22.004733-04:00
1,localhost,laurel,type=EXECVE msg=audit(1720451679.133:2225): ar...,info,local0,2024-07-09T12:21:22.004745-04:00
2,localhost,laurel,"type=CWD msg=audit(1720451679.133:2225): cwd=""...",info,local0,2024-07-09T12:21:22.004746-04:00
3,localhost,laurel,type=PATH msg=audit(1720451679.133:2225): item...,info,local0,2024-07-09T12:21:22.004748-04:00
4,localhost,laurel,type=PATH msg=audit(1720451679.133:2225): item...,info,local0,2024-07-09T12:21:22.004750-04:00


In [5]:
def normalize(df, col):
    if col not in df.columns:
        print(f"Error, column {col} not in df")
        return None
    df_expanded = pd.json_normalize(df[col])
    #print(df_expanded.head())
    df = df.drop(col, axis=1)
    df = df.join(df_expanded, rsuffix=f'_{col}')
    return df

In [6]:
df.columns

Index(['hostname', 'ident', 'msg', 'severity', 'facility', 'timereported'], dtype='object')

In [7]:
df

,hostname,ident,msg,severity,facility,timereported
0,localhost,laurel,type=SYSCALL msg=audit(1720451679.133:2225): a...,info,local0,2024-07-09T12:21:22.004733-04:00
1,localhost,laurel,type=EXECVE msg=audit(1720451679.133:2225): ar...,info,local0,2024-07-09T12:21:22.004745-04:00
2,localhost,laurel,"type=CWD msg=audit(1720451679.133:2225): cwd=""...",info,local0,2024-07-09T12:21:22.004746-04:00
3,localhost,laurel,type=PATH msg=audit(1720451679.133:2225): item...,info,local0,2024-07-09T12:21:22.004748-04:00
4,localhost,laurel,type=PATH msg=audit(1720451679.133:2225): item...,info,local0,2024-07-09T12:21:22.004750-04:00
...,...,...,...,...,...,...
995,localhost,laurel,type=SYSCALL msg=audit(1720536710.308:27): arc...,info,local0,2024-07-09T12:21:22.011464-04:00
996,localhost,laurel,type=EXECVE msg=audit(1720536710.308:27): argc...,info,local0,2024-07-09T12:21:22.011467-04:00
997,localhost,laurel,"type=CWD msg=audit(1720536710.308:27): cwd=""/""",info,local0,2024-07-09T12:21:22.011468-04:00
998,localhost,laurel,type=PATH msg=audit(1720536710.308:27): item=0...,info,local0,2024-07-09T12:21:22.011470-04:00


In [8]:
df = normalize(df, 'msg')
print(df.head())
df = normalize(df, 'CWD')
df = normalize(df, 'PATH')
df = normalize(df, 'SYSCALL')
df = normalize(df, 'PROCTITLE')
df = normalize(df, 'EXECVE')
df = normalize(df, 0)
df = normalize(df, 1)
df.head()

df['ARGV_PROCTITLE_str'] = df['ARGV_PROCTITLE'].apply(lambda x: ' '.join(x))

df['unix_time'] = pd.to_numeric(df['ID'].str.split(':').str[0])
df['timedelta'] = df['unix_time'].diff()
df['time'] = pd.to_datetime(df['unix_time'], unit='s')

df['ppid_time'] = pd.to_datetime(pd.to_numeric(df['PPID.EVENT_ID'].str.split(':').str[0]), unit='s')
df['ppid_timedelta'] = df['ppid_time'].diff()

df['pid_time'] = pd.to_datetime(pd.to_numeric(df['PID.EVENT_ID'].str.split(':').str[0]), unit='s')
df['pid_timedelta'] = df['pid_time'].diff()

df = df.drop(['ID', 'PID.EVENT_ID', 'PPID.EVENT_ID', 'unix_time', 'ARGV_PROCTITLE', 'time', 'pid_time', 'ppid_time'], axis=1)

    hostname   ident severity facility                      timereported
0  localhost  laurel     info   local0  2024-07-09T12:21:22.004733-04:00
1  localhost  laurel     info   local0  2024-07-09T12:21:22.004745-04:00
2  localhost  laurel     info   local0  2024-07-09T12:21:22.004746-04:00
3  localhost  laurel     info   local0  2024-07-09T12:21:22.004748-04:00
4  localhost  laurel     info   local0  2024-07-09T12:21:22.004750-04:00
Error, column CWD not in df


AttributeError: 'NoneType' object has no attribute 'columns'

In [ ]:
for col in df.columns:
    print(f'{col} : {df.iloc[1][col]}')

In [ ]:
columns = ['auid', 'uid', 'gid', 'euid', 'suid', 'fsuid', 'egid', 'sgid', 'fsgid']

def check_row_id_num(row):
    return pd.Series({col: row[col] == 0 for col in columns})

checks = df.apply(check_row_id_num, axis=1)
df['num_id_anomalies'] = checks.sum(axis=1)
df = df.drop(columns, axis=1)

In [ ]:
columns = ['AUID', 'UID', 'GID', 'EUID', 'SUID', 'FSUID', 'EGID', 'SGID', 'FSGID']

# Use apply to efficiently check values for each row
def check_id_row(row):    
    return pd.Series({col: int(not(row[col] in row['UID_GROUPS'])) for col in columns})

checks = df.apply(check_id_row, axis=1)

df['id_anomalies'] = checks.sum(axis=1)
df = df.drop(columns, axis=1)
df = df.drop('UID_GROUPS', axis=1)

In [ ]:
for col in df.columns:
    print(f'{col} : {df.iloc[1][col]}')

In [ ]:
# filter final columns
columns_to_keep = ['cwd', 'exit', 'items', 'ppid', 'pid', 'comm', 'timedelta', 'pid_timedelta', 'ppid_timedelta', 'id_anomalies', 'num_id_anomalies', 'ARGV_PROCTITLE_str']
df = df [columns_to_keep]

In [ ]:
df.iloc[1]

In [ ]:
os.getcwd()

In [ ]:
# Save this df to file
save_fp = '../data/laurel_anomalous_new/save1.csv'
df.to_csv(save_fp, index=False)

In [ ]:
# Final Preprocessing Function
def preprocess_laurel(df):
    
    df = normalize(df, 'CWD')
    df = normalize(df, 'PATH')
    df = normalize(df, 'SYSCALL')
    df = normalize(df, 'PROCTITLE')
    df = normalize(df, 'EXECVE')
    df = normalize(df, 0)
    df = normalize(df, 1)
    print(df.head())

    columns_to_keep = ['cwd', 'exit', 'items', 'ppid', 'pid', 'comm', 'timedelta', 'pid_timedelta', 'ppid_timedelta', 'id_anomalies', 'num_id_anomalies', 'ARGV_PROCTITLE_str']
    columns_to_drop = ['ID', 'PID.EVENT_ID', 'PPID.EVENT_ID', 'unix_time', 'ARGV_PROCTITLE', 'time', 'pid_time', 'ppid_time']
    
    try:
        df['ARGV_PROCTITLE_str'] = df['ARGV_PROCTITLE'].apply(lambda x: ' '.join(x))
    except:
        pass
    
    df['unix_time'] = pd.to_numeric(df['ID'].str.split(':').str[0])
    df['timedelta'] = df['unix_time'].diff()
    df['time'] = pd.to_datetime(df['unix_time'], unit='s')

    try:
        df['ppid_time'] = pd.to_datetime(pd.to_numeric(df['PPID.EVENT_ID'].str.split(':').str[0]), unit='s')
        df['ppid_timedelta'] = df['ppid_time'].diff()
        columns_to_keep.remove('ppid_timedelta')
        columns_to_drop.remove('ppid_time')
    except:
        pass
    
    df['pid_time'] = pd.to_datetime(pd.to_numeric(df['PID.EVENT_ID'].str.split(':').str[0]), unit='s')
    df['pid_timedelta'] = df['pid_time'].diff()
    
    df = df.drop(columns_to_drop, axis=1)

    columns = ['auid', 'uid', 'gid', 'euid', 'suid', 'fsuid', 'egid', 'sgid', 'fsgid']
    
    def check_row_id_num(row):
        return pd.Series({col: row[col] == 0 for col in columns})
    
    checks = df.apply(check_row_id_num, axis=1)
    df['num_id_anomalies'] = checks.sum(axis=1)
    df = df.drop(columns, axis=1)

    columns = ['AUID', 'UID', 'GID', 'EUID', 'SUID', 'FSUID', 'EGID', 'SGID', 'FSGID']
    
    # Use apply to efficiently check values for each row
    def check_id_row(row):    
        return pd.Series({col: int(not(row[col] in row['UID_GROUPS'])) for col in columns})
    
    checks = df.apply(check_id_row, axis=1)
    
    df['id_anomalies'] = checks.sum(axis=1)
    df = df.drop(columns, axis=1)
    df = df.drop('UID_GROUPS', axis=1)

    # filter final columns
    df = df [columns_to_keep]

    return df

In [ ]:
def read(fp):
    data_list = []
    lines_limit = 1000 # limit reading lines, done for memory constraints
    i = 0
    
    with open(fp, 'r') as file:
        for line in file:
            i+=1
            if i > lines_limit:
                break
                # Find the index of the first '{'
            index = line.find('{')
            if index != -1:
                json_data = line[index:]
                data = json.loads(json_data)
                data_list.append(data)
                    
    df = pd.DataFrame(data_list)
    return df

In [ ]:
def save(df, save_fp):
    # Save this df to file
    #save_fp = '../data/laurel_anomalous_new/save1.csv'
    df.to_csv(save_fp, index=False)

In [ ]:
for fp in fps:
    df = read(fp)
    df = preprocess_laurel(df)
    save(df, fp + '.csv')